In [1]:
require 'nn';

## define your own layer

In [2]:
CSplit, parent = torch.class('nn.CSplit', 'nn.Module')

function CSplit:__init(firstCount)
    self.firstCount = firstCount
    parent.__init(self)
end

function CSplit:updateOutput(input)
    local inputSize = input:size()[1]
    local firstCount = self.firstCount
    local secondCount = inputSize - firstCount
    local first = torch.Tensor(self.firstCount)
    local second = torch.Tensor(secondCount)
    for i=1, inputSize do
        if i <= firstCount then
            first[i] = input[i]
        else
            second[i - firstCount] = input[i]
        end
    end
    self.output = {first, second}
    return self.output
end

function CSplit:updateGradInput(input, gradOutput)    
    local inputSize = input:size()[1]
    self.gradInput = torch.Tensor(input)
    for i=1, inputSize do
        if i <= self.firstCount then
            self.gradInput[i] = gradOutput[1][i]
        else
            self.gradInput[i] = gradOutput[2][i-self.firstCount]
        end
    end
    return self.gradInput
end

In [3]:
testNet = nn.CSplit(4)

In [4]:
input = torch.randn(10)
output = testNet:forward(input)
print(input)
print(output[1])
print(output[2])
testNet:backward(input, {torch.randn(4), torch.randn(6)})

 0.1294
-0.5742
 1.8099
 0.5352
 0.5399
-1.0563
-0.6766
 1.5584
 1.4493
 0.4544
[torch.DoubleTensor of size 10]

 0.1294
-0.5742
 1.8099
 0.5352
[torch.DoubleTensor of size 4]

 0.5399
-1.0563
-0.6766
 1.5584
 1.4493
 0.4544
[torch.DoubleTensor of size 6]



## test the layer in real network

### build and test the network using CSplit

In [5]:
net = nn.Sequential()
net:add(nn.Reshape(1*28*28))
net:add(nn.Linear(1*28*28, 50))
net:add(nn.ReLU())
net:add(nn.CSplit(25))

parallelNet = nn.ParallelTable()
firstPartNet = nn.Linear(25, 10)
secondPartNet = nn.Linear(25, 10)
parallelNet:add(firstPartNet)
parallelNet:add(secondPartNet)
net:add(parallelNet)
net:add(nn.JoinTable(1))
net:add(nn.ReLU())
net:add(nn.Linear(20, 10))
-- net:add(nn.Dropout(0.5))
net:add(nn.LogSoftMax())

output = net:forward(torch.randn(28, 28))
net:backward(torch.randn(28, 28), torch.randn(10))
print(output)

-2.4695
-2.2748
-2.1606
-2.2838
-2.1932
-2.7600
-2.3625
-2.2614
-2.2657
-2.1336
[torch.DoubleTensor of size 10]



### define criterion

In [6]:
crit = nn.ClassNLLCriterion()

### training the network with mnist dataset

#### load dataset and preprocessing

In [7]:
-- install mnist package according https://github.com/andresy/mnist 
local mnist = require 'mnist'
local train = mnist.traindataset()
Xt = train.data:double()
Yt = train.label
local test = mnist.testdataset()
Xv = test.data:double()
Yv = test.label
Yt[Yt:eq(0)] = 10
Yv[Yv:eq(0)] = 10
itorch.image(Xt:narrow(1, 1, 6))

In [8]:
print(Xt[1]:type())
testData = torch.randn(28*28)
print(testData:type())
output = net:forward(Xt[1])
print(output)

torch.DoubleTensor
torch.DoubleTensor
-3.2954e+01
-2.2430e+01
-2.7157e+01
-1.4346e+01
-1.1874e-06
-5.3523e+01
-3.2618e+01
-1.7043e+01
-1.4402e+01
-1.9547e+01
[torch.DoubleTensor of size 10]



#### set arguments

In [9]:
epoch = 10
batchsize = 128
lr = 1e-3

#### train and test after each epoch training

In [10]:
counter = 1
lossTb = {}
testLossTb = {}
counterTb = {}
parameters, gradParameters = net:getParameters()
for i=1, epoch do    
    for bi=1, Xt:size()[1]/batchsize do
        tmpGradParameters = torch.zeros(gradParameters:size())
        avgLoss = 0
        for bj=1, batchsize do
            j = (bi-1) * batchsize + bj
            output = net:forward(Xt[j])
            loss = crit:forward(output, Yt[j])
            avgLoss = avgLoss + loss
            critBack = crit:backward(output, Yt[j])
            gradParameters:zero()
            net:backward(Xt[j], critBack)
            tmpGradParameters:add(gradParameters)
        end
        tmpGradParameters:div(batchsize)
        avgLoss = avgLoss / batchsize
        parameters:add(-1*lr*tmpGradParameters)
        counter = counter + 1
        if counter % 10 == 0 then
            table.insert(lossTb, avgLoss)
            table.insert(counterTb, counter)
        end
        if counter % 100 == 0 then
            print(string.format("traning loss %.2f, counter:%08d", avgLoss, counter))
        end
    end
    wrongCount = 0
    for j=1, Xv:size()[1] do
        output = net:forward(Xv[j])        
        maxIdx = 1
        maxNum = output[1]
        for k=1, output:size()[1] do
            if output[k] > maxNum then
                maxNum = output[k]
                maxIdx = k
            end
        end
        if maxIdx ~= Yv[j] then
            wrongCount = wrongCount + 1
        end
    end
    wrongRate = wrongCount / Xv:size()[1]
    print(string.format("wrong rate %.2f, counter:%08d", wrongRate, counter))
end
print("traning ends")

traning loss 1.31, counter:00000100


traning loss 0.43, counter:00000200


traning loss 0.68, counter:00000300


traning loss 0.35, counter:00000400


wrong rate 0.14, counter:00000469


...

traning loss 0.09, counter:00004300


traning loss 0.21, counter:00004400


traning loss 0.04, counter:00004500


traning loss 0.27, counter:00004600


wrong rate 0.06, counter:00004681
traning ends


### visualization

In [14]:
Plot = require 'itorch.Plot';
plot = Plot():line(counterTb, lossTb, "blue", "training loss")
plot:title("training loss")
plot:draw()